In [ ]:
!git clone https://github.com/neurowelt/AnimateDiff.git
!pip install einops omegaconf safetensors diffusers transformers==4.30.2 xformers triton==2.0.0 accelerate controlnet-aux==0.0.6
!apt -y install -qq aria2
!rm -rf /content/AnimateDiff/models/StableDiffusion
!git clone -b fp16 https://huggingface.co/runwayml/stable-diffusion-v1-5 /content/AnimateDiff/models/StableDiffusion/
!git clone https://huggingface.co/lllyasviel/control_v11p_sd15_softedge /content/AnimateDiff/models/Controlnet/control_v11p_sd15_softedge
# !git clone https://huggingface.co/lllyasviel/control_v11f1p_sd15_depth /content/AnimateDiff/models/Controlnet/control_v11f1p_sd15_depth
# !git clone https://huggingface.co/lllyasviel/control_v11p_sd15_canny /content/AnimateDiff/models/Controlnet/control_v11p_sd15_canny

In [ ]:
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v14.ckpt -d /content/animatediff/models/Motion_Module -o mm_sd_v14.ckpt
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v15.ckpt -d /content/animatediff/models/Motion_Module -o mm_sd_v15.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/mm_sd_v15_v2.ckpt -d /content/AnimateDiff/models/Motion_Module -o mm_sd_v15_v2.ckpt
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/toonyou_beta3.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o toonyou_beta3.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/CounterfeitV30_v30.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o CounterfeitV30_v30.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/FilmVelvia2.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o FilmVelvia2.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/Pyramid%20lora_Ghibli_n3.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o Pyramid%20lora_Ghibli_n3.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/TUSUN.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o TUSUN.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/lyriel_v16.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o lyriel_v16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/majicmixRealistic_v5Preview.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o majicmixRealistic_v5Preview.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/moonfilm_filmGrain10.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o moonfilm_filmGrain10.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/moonfilm_reality20.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o moonfilm_reality20.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/rcnzCartoon3d_v10.safetensors -d /content/animatediff/models/DreamBooth_LoRA -o rcnzCartoon3d_v10.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/AnimateDiff/resolve/main/realisticVisionV20_v20.safetensors -d /content/AnimateDiff/models/DreamBooth_LoRA -o realisticVisionV20_v20.safetensors

In [ ]:
%cd /content/AnimateDiff

In [ ]:
import os
import argparse
import datetime
import inspect
from pathlib import Path
from types import SimpleNamespace
from typing import Optional

try:
    import accelerate
except ImportError:
    print("No accelerate")

import torch
from PIL import Image
from omegaconf import OmegaConf

from diffusers import AutoencoderKL, DDIMScheduler

from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer

from animatediff.models.unet import UNet3DConditionModel
from animatediff.pipelines.pipeline_animation import AnimationPipeline
from animatediff.utils.util import save_videos_grid, load_weights, tensor2vid
from diffusers.utils.import_utils import is_xformers_available

device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
config = SimpleNamespace(
    L = 16,
    W = 512,
    H = 512,
    config = "configs/prompts/v2/5-RealisticVision.yaml",
    inference_config = "configs/inference/inference-v2.yaml",
    pretrained_model_path = "models/StableDiffusion",
    motion_module = "models/Motion_Module/mm_sd_v15_v2.ckpt",
    prompt = ["diamonds and gold jewels, fire lava colors"],
    n_prompt = [""],
    seed = [1],
    steps = 25,
    guidance_scale = 7.5,
    control = SimpleNamespace(
        video_path = "./input.mp4", # smiling, dance or your video
        get_each = 1, # get each frame from video
        conditioning_scale = 0.75, # controlnet strength
        controlnet_processor = "softedge", # softedge, canny, depth
        controlnet_pipeline = "models/StableDiffusion",
        controlnet_processor_path = "models/Controlnet/control_v11p_sd15_softedge", # control_v11p_sd15_softedge, control_v11f1p_sd15_depth, control_v11p_sd15_canny
        guess_mode = True
    )
)

savedir = "results"

In [4]:
if not os.path.exists(savedir):
    os.makedirs(savedir)

samples = []
sample_idx = 0

motion_module = config.motion_module
inference_config = OmegaConf.load(getattr(config, "inference_config", None))

In [ ]:
### >>> create validation pipeline >>> ###
tokenizer    = CLIPTokenizer.from_pretrained(config.pretrained_model_path, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(config.pretrained_model_path, subfolder="text_encoder")
vae          = AutoencoderKL.from_pretrained(config.pretrained_model_path, subfolder="vae")
unet         = UNet3DConditionModel.from_pretrained_2d(config.pretrained_model_path, subfolder="unet", unet_additional_kwargs=OmegaConf.to_container(inference_config.unet_additional_kwargs))

In [ ]:
if is_xformers_available():
    unet.enable_xformers_memory_efficient_attention()

pipeline = AnimationPipeline(
    vae=vae, text_encoder=text_encoder, tokenizer=tokenizer, unet=unet,
    scheduler=DDIMScheduler(**OmegaConf.to_container(inference_config.noise_scheduler_kwargs)),
).to(device)

In [ ]:
pipeline, controlnet, down_features, mid_features = load_weights(
    pipeline,
    config,
    device,
    # motion module
    motion_module_path = motion_module,
    motion_module_lora_configs = getattr(config, "motion_module_lora_configs", []),
    # image layers
    dreambooth_model_path = getattr(config, "dreambooth_path", ""),
    lora_model_path = getattr(config, "lora_model_path", ""),
    lora_alpha = getattr(config, "lora_alpha", 0.8),
)

In [ ]:
import time

prompts = getattr(config, "prompt", None)
if not prompts:
    raise ValueError("No prompts provided")
n_prompts = list(config.n_prompt) * len(prompts) if len(config.n_prompt) == 1 else config.n_prompt

random_seeds = getattr(config, "seed", [-1])
random_seeds = [random_seeds] if isinstance(random_seeds, int) else list(random_seeds)
random_seeds = random_seeds * len(prompts) if len(random_seeds) == 1 else random_seeds

init_image = None

for prompt_idx, (prompt, n_prompt, random_seed) in enumerate(zip(prompts, n_prompts, random_seeds)):

    # manually set random seed for reproduction
    if random_seed != -1: torch.manual_seed(random_seed)
    else: torch.seed()

    if controlnet is not None:
        down_features, mid_features = controlnet(config.control.video_path, prompt, n_prompt, random_seed)
        torch.cuda.empty_cache()
        time.sleep(2)

    print(f"current seed: {torch.initial_seed()}")
    print(f"sampling {prompt} ...")
    sample = pipeline(
        prompt,
        negative_prompt     = n_prompt,
        num_inference_steps = config.steps,
        guidance_scale      = config.guidance_scale,
        width               = config.W,
        height              = config.H,
        video_length        = config.L,
        init_image          = init_image,
        down_block_control  = down_features,
        mid_block_control   = mid_features,
    ).videos
    samples.append(sample)

    # Continue from last frame
    if getattr(config, "C", False):
        frames = tensor2vid(sample)
        init_image = Image.fromarray(frames[-1]).convert("RGB")

    prompt = "-".join((prompt.replace("/", "").split(" ")[:10]))
    save_videos_grid(sample, f"{savedir}/sample/{sample_idx}-{prompt}.gif")
    print(f"save to {savedir}/sample/{prompt}.gif")

    sample_idx += 1
    torch.cuda.empty_cache()
    time.sleep(2)